# <font color='red'>1.0 IMPORT</font>

In [54]:
# ENVIRONMENT

# !pip install mysql-connector-python
# !pip install pymysql
# !pip install psycopg2
# !pip install sqlalchemy

import pandas as pd
import time
from datetime import datetime
import mysql.connector
import numpy as np
from sqlalchemy import create_engine

# pd.options.display.max_rows = 2000
# pd.options.display.width = 120
# pd.options.display.max_colwidth = 100

ETL_VERSAO = 'etl_v1.0'

# <font color='red'>2.0 EXTRACT</font>

In [3]:
def LeFontes(strAut):

    print('Iniciando leitura de dados nativos...', end='')
    
    sql_form = (
    "SELECT * "
    "FROM form"
    )
    sql_tasks = (
    "SELECT DISTINCT `Protocolo`, `Entidade`, `Serviço`, `Usuário`, `Grupo`, `Data e Hora de conclusão`, "
    "`Data e Hora de criação`, `Ação`, `Encaminhado para`, `Processo encerrado`, `Processo cancelado`, "
    "`Motivo de cancelamento`, `Status externo`, `Categoria`, `Grupo responsável`, `Prazo (em segundos)` "
    "FROM tasks"
    )
    sql_sla = (
    "SELECT * "
    "FROM sla"
    )
    sql_rating = (
    "SELECT * "
    "FROM rating"
    )
    
    try:
        connection = mysql.connector.connect(host     = strAut['My_host'], 
                                             database = strAut['My_db'], 
                                             user     = strAut['My_user'], 
                                             password = strAut['My_pw'])

        df_int_tasks =   pd.read_sql(sql_tasks,  con=connection)
        df_int_sla =     pd.read_sql(sql_sla,    con=connection)
        df_int_form =    pd.read_sql(sql_form,   con=connection)
        df_int_rating =  pd.read_sql(sql_rating, con=connection)
        
        print('OK')

    except mysql.connector.Error as error:
        print("Failed to read record from MySQL table {}".format(error))

    finally:
        if (connection.is_connected()):
            connection.close()
            print(f'tasks:{df_int_tasks.shape[0]} registros lidos em {df_int_tasks.shape[1]} colunas')
            print(f'sla: {df_int_sla.shape[0]} registros lidos em {df_int_sla.shape[1]} colunas')
            print(f'form: {df_int_form.shape[0]} registros lidos em {df_int_form.shape[1]} colunas')
            print(f'rating: {df_int_rating.shape[0]} registros lidos em {df_int_rating.shape[1]} colunas')
            print("MySQL connection is closed")
            
    return df_int_tasks, df_int_sla, df_int_form, df_int_rating

# <font color='red'>3.0 TRANSFORM</font>

## <font color='blue'>trata os datasets</font>

### <font color='black'>trata o dataset sla</font>

In [4]:
def trSLA(df):
    
    print('df_sla - iniciando transformação... ', end='')
    
    MINUTOS_PARA_DIAS = 1440 # quantidade de minutos em um dia

    # preenche com null as colunas que contém 'null' como tipo string
    df.loc[(df['limiteMinimo'] == ''), 'limiteMinimo'] = np.nan
    df.loc[(df['limiteMaximo'] == ''), 'limiteMaximo'] = np.nan

    # transforma o tipo de coluna para float
    df['limiteMinimo'] = df['limiteMinimo'].astype(float)
    df['limiteMaximo'] = df['limiteMaximo'].astype(float)

    # transforma a unidade de medida de minuto para dias
    df['limiteMinimo'] = df['limiteMinimo'] / MINUTOS_PARA_DIAS
    df['limiteMaximo'] = df['limiteMaximo'] / MINUTOS_PARA_DIAS

    # preenche com valores extremos os limites máximos e limites mínimos
    df.loc[(df['limiteMinimo'].isna()), 'limiteMinimo'] = -9999999.9
    df.loc[(df['limiteMaximo'].isna()), 'limiteMaximo'] = 9999999.9

    # cria mais uma coluna de status para pivotar limite mínimo e limite máxio
    df['status2'] = df['status']
    df.rename(columns={'status': 'StatusLmin', 'status2': 'StatusLmax'}, inplace = True)
    
    # faz pivot da coluna StatusLmin
    idx = ['entityCode', 'service', 'StatusLmax', 'limiteMaximo']
    df = df.pivot(columns = 'StatusLmin', values = 'limiteMinimo', index=idx).reset_index()
    df.columns.name = None
    dic_renome = {'Dentro do prazo' : 'Dentro do Prazo LMin', 
                  'Fora do prazo' : 'Fora do Prazo LMin',
                  'Perto do prazo' : 'Perto do Prazo LMin'}
    df.rename(columns=dic_renome, inplace = True)

    # faz pivot da coluna StatusLmax
    idx = ['entityCode', 'service', 'Dentro do Prazo LMin', 'Fora do Prazo LMin', 'Perto do Prazo LMin']
    df = df.pivot(columns = 'StatusLmax', values = 'limiteMaximo', index=idx).reset_index()
    df.columns.name = None
    dic_renome = {'Dentro do prazo' : 'Dentro do Prazo LMax', 
                  'Fora do prazo' : 'Fora do Prazo LMax',
                  'Perto do prazo' : 'Perto do Prazo LMax'}
    df.rename(columns=dic_renome, inplace = True)

    # agrupa por entidade e serviço
    df = df.groupby('service').sum().reset_index()

    print('%d linhas OK.' %df.shape[0])
    
    return df

### <font color='black'>trata o dataset form</font>

In [5]:
def trForm(df):
    
    print('df_form - iniciando transformação... ', end='')

    df.columns = ['atributo', 'nome', 'valor', 'protocolo', 'servico', 'tipo', 'campo_relacionado']
    df['atributo'] = df['atributo'].str.lower()
    
    print('%d linhas OK.' %df.shape[0])

    return df

### <font color='black'>trata o dataset rating</font>

In [6]:
def trRating(df):

    print('df_rating - iniciando transformação... ', end='')

    # renomeia as colunas
    cols = ['Solicitacao', 'NotaAvaliacao', 'MotivoAvaliacao', 'DataHoraAvaliacao']
    df.columns = cols

    # separa e formata as colunas de datas e horas 
    df['DataAvaliacao'] = pd.to_datetime(df['DataHoraAvaliacao']).dt.date
    df['HoraAvaliacao'] = pd.to_datetime(df['DataHoraAvaliacao']).dt.time

    # deleta a coluna que contém data e hora
    df.drop(['DataHoraAvaliacao'], axis=1, inplace=True)

    # preenche colunas de linhas vazias
    df.loc[df['MotivoAvaliacao'] == '', 'MotivoAvaliacao'] = '<motivo vazio>'
    
    print('%d linhas OK.' %df.shape[0])
    
    return df

### <font color='black'>trata o dataset tasks</font>

In [7]:
def trTasks(df):

    print('df_tasks - iniciando transformação... ', end='')

    SEM_STATUS = '<sem status inicial>'

    # renomeia as colunas
    lst_colunas_tasks = ['Protocolo', 'Entidade', 'Servico', 'Usuarios','Grupo', 'DataHora_Conclusao',
                         'DataHora_Criacao', 'Acao', 'EncaminhadoPara', 'ProcessoEncerrado', 'ProcessoCancelado',
                         'MotivoCancelamento', 'StatusExterno', 'Categoria', 'GrupoResponsavel','Prazo']
    df.columns = lst_colunas_tasks

    # coloca null em todas as colunas com dados vazios ou null
    df = df.replace(['null', '', '-'], np.nan)

    # coloca dataset em ordem para acertar o status externo
    lst = ['Entidade', 'Protocolo', 'DataHora_Criacao']
    df = df.sort_values(by=lst).reset_index(drop=True)

    # acerta os tipos das colunas datetime
    df['DataHora_Conclusao'] = df['DataHora_Conclusao'].astype('datetime64')
    df['DataHora_Criacao']   = df['DataHora_Criacao'].astype('datetime64')

    # rotina para preencher status externo vazio
    # pega sempre o anterior e se o primeiro status estiver vazio coloca "sem status inicial"
    if pd.isna(df.loc[0, 'StatusExterno']):
        df.loc[0, 'StatusExterno'] = SEM_STATUS

    for i in range(1, df.shape[0]):
        if pd.isna(df.loc[i, 'StatusExterno']):    
            if df.loc[i, 'Protocolo'] == df.loc[i - 1, 'Protocolo']:
                df.loc[i, 'StatusExterno'] = df.loc[i - 1, 'StatusExterno']
            else:
                df.loc[i, 'StatusExterno'] = SEM_STATUS

    print('%d linhas OK.' %df.shape[0])

    return df

## <font color='blue'>monta tabelas dimensões</font>

### DIM acoes

In [8]:
def Mt_dim_Acoes(df):

    print('dim_Acoes - iniciando montagem... ', end='')

    # trasnforma a coluna Acao em uma coluna do tipo "category'
    df['tmpAcao'] = df['Acao'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Ação
    df['FK_dim_Acoes'] = df['tmpAcao'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_Acoes', 'Acao']].drop_duplicates()
    dfx.rename(columns={'FK_dim_Acao': 'PK_dim_Acao'}, inplace = True)
    
    # preenche colunas com linhas vazias
    dfx.loc[dfx['Acao'] == '', ['Acao']] = '<sem ação determinada>'
    
    # exclui a coluna Acoes de df_tasks que será a futura tabela fato
    df.drop(['Acao', 'tmpAcao'], axis=1, inplace=True)
    
    print('%d linhas OK.' %dfx.shape[0])

    return dfx

### DIM categoria servicos

In [9]:
def Mt_dim_CategoriaServico(df):

    print('dim_Categoria - iniciando montagem... ', end='')

    # trasnforma a coluna Categoria em uma coluna do tipo "category'
    df['tmpCategoria'] = df['Categoria'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão CategoriasServicos
    df['FK_dim_CategoriasServicos'] = df['tmpCategoria'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_CategoriasServicos', 'Categoria']].drop_duplicates()
    dfx.rename(columns={'FK_dim_CategoriasServicos': 'PK_dim_CategoriasServicos'}, inplace = True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['Categoria'] == '', ['Categoria']] = '<categoria indefinida>'
    
    # exclui a coluna Categoria de df_tasks que será a futura tabela fato
    df.drop(['Categoria', 'tmpCategoria'], axis=1, inplace=True)
    
    print('%d linhas OK.' %dfx.shape[0])

    return dfx

### DIM encaminhamento

In [10]:
def Mt_dim_Encaminhamento(df):

    print('dim_Encaminhamento - iniciando montagem... ', end='')

    # trasnforma a coluna EncaminhadoPara em uma coluna do tipo "category'
    df['tmpEncaminhadoPara'] = df['EncaminhadoPara'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Encaminhamento
    df['FK_dim_Encaminhamento'] = df['tmpEncaminhadoPara'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_Encaminhamento', 'EncaminhadoPara']].drop_duplicates()
    dfx.rename(columns={'FK_dim_Encaminhamento': 'PK_dim_Encaminhamento'}, inplace = True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['EncaminhadoPara'] == '', ['EncaminhadoPara']] = '<sem encaminhamento>'

    # exclui a coluna Entidade de df_tasks que será a futura tabela fato
    df.drop(['EncaminhadoPara', 'tmpEncaminhadoPara'], axis=1, inplace=True)

    print('%d linhas OK.' %dfx.shape[0])

    return dfx

### DIM entidade

In [11]:
def Mt_dim_Entidade(df):

    print('dim_Entidade - iniciando montagem... ', end='')

    # trasnforma a coluna Entidade em uma coluna do tipo "category'
    df['tmpEntidade'] = df['Entidade'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Entidades
    df['FK_dim_Entidades'] = df['tmpEntidade'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_Entidades', 'Entidade']].drop_duplicates()
    dfx['Entidade'] = dfx['Entidade'].str.upper()
    dfx.rename(columns={'FK_dim_Entidades': 'PK_dim_Entidades'}, inplace = True)

    # exclui a coluna Entidade de df_tasks que será a futura tabela fato
    df.drop(['Entidade', 'tmpEntidade'], axis=1, inplace=True)

    print('%d linhas OK.' %dfx.shape[0])

    return dfx

### DIM grupo responsavel

In [12]:
def Mt_dim_GrupoResponsavel(df):

    print('dim_GrupoResponsavel - iniciando montagem... ', end='')

    # trasnforma a coluna GrupoResponsavel em uma coluna do tipo "category'
    df['tmpGrupoResponsavel'] = df['GrupoResponsavel'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão GrupoResponsavel
    df['FK_dim_GrupoResponsavel'] = df['tmpGrupoResponsavel'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_GrupoResponsavel', 'GrupoResponsavel']].drop_duplicates()
    dfx.rename(columns={'FK_dim_GrupoResponsavel': 'PK_dim_GrupoResponsavel'}, inplace = True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['GrupoResponsavel'] == '', ['GrupoResponsavel']] = '<grupo responsável não definido>'

    # exclui a coluna GrupoResponsavel de df_tasks que será a futura tabela fato
    df.drop(['GrupoResponsavel', 'tmpGrupoResponsavel'], axis=1, inplace=True)
    
    print('%d linhas OK.' %dfx.shape[0])
    
    return dfx

### DIM grupo usuarios

In [13]:
def Mt_dim_GruposUsuarios(df):

    print('dim_GruposUsuarios - iniciando montagem... ', end='')

    # trasnforma a coluna Grupo em uma coluna do tipo "category'
    df['tmpGrupo'] = df['Grupo'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Grupo
    df['FK_dim_GruposUsuarios'] = df['tmpGrupo'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_GruposUsuarios', 'Grupo']].drop_duplicates()
    dfx.rename(columns={'FK_dim_GruposUsuarios': 'PK_dim_GruposUsuarios'}, inplace = True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['Grupo'] == '', ['Grupo']] = '<grupo usuário não definido>'

    # exclui a coluna Grupo de df_tasks que será a futura tabela fato
    df.drop(['Grupo', 'tmpGrupo'], axis=1, inplace=True)
    
    print('%d linhas OK.' %dfx.shape[0])
    
    return dfx

### DIM motivos cancelamento

In [14]:
def Mt_MotivosCanc(df):

    print('dim_MotivosCanc - iniciando montagem... ', end='')

    # trasnforma a coluna MotivoCancelamento em uma coluna do tipo "category'
    df['tmpMotivoCancelamento'] = df['MotivoCancelamento'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão MotivoCanc
    df['FK_dim_MotivosCanc'] = df['tmpMotivoCancelamento'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_MotivosCanc', 'MotivoCancelamento']].drop_duplicates()
    dfx.rename(columns={'FK_dim_MotivosCanc': 'PK_dim_MotivosCanc'}, inplace = True)


    # preenche colunas com linhas vazias
    dfx.loc[dfx['MotivoCancelamento'] == '', ['MotivoCancelamento']] = '<sem motivo de cancelamento>'

    # exclui a coluna MotivoCancelamento de df_tasks que será a futura tabela fato
    df.drop(['MotivoCancelamento', 'tmpMotivoCancelamento'], axis=1, inplace=True)

    print('%d linhas OK.' %dfx.shape[0])
    
    return dfx

### DIM servico

In [15]:
def Mt_Servico(df, dff_sla):

    print('dim_Servico - iniciando montagem... ', end='')

    # trasnforma a coluna Servico em uma coluna do tipo "category'
    df['tmpServico'] = df['Servico'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Servico
    df['FK_dim_Servicos'] = df['tmpServico'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_Servicos', 'Servico']].drop_duplicates()

    # exclui a coluna Servico de df_tasks que será a futura tabela fato
    df.drop(['Servico', 'tmpServico'], axis=1, inplace=True)

    # defive os nomes das colunas de SLA
    cols = {'FK_dim_Servicos': 'PK_dim_Servicos',
            'Dentro do Prazo LMin': 'sla_VD_Lmin',
            'Perto do Prazo LMin': 'sla_AM_Lmin',
            'Fora do Prazo LMin': 'sla_VM_Lmin',
            'Dentro do Prazo LMax': 'sla_VD_Lmax',
            'Perto do Prazo LMax': 'sla_AM_LMax',
            'Fora do Prazo LMax': 'sla_VM_LMax',
           }
    # faz um merge da dimensão dim_Servicos com a tabela de SLAs
    dfx = dfx.merge(dff_sla, left_on='Servico', right_on='service', how='left')
    dfx = dfx.drop('service', axis=1)
    dfx.rename(columns=cols, inplace = True)
    
    print('%d linhas OK.' %dfx.shape[0])
    
    return dfx

### DIM status externo

In [16]:
def Mt_StatusExt(df):

    print('dim_StatusExt - iniciando montagem... ', end='')

    # trasnforma a coluna StatusExterno em uma coluna do tipo "category'
    df['tmpStatusExterno'] = df['StatusExterno'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Grupo
    df['FK_dim_StatusExt'] = df['tmpStatusExterno'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_StatusExt', 'StatusExterno']].drop_duplicates()
    dfx.rename(columns={'FK_dim_StatusExt': 'PK_dim_StatusExt'}, inplace = True)

    # exclui a coluna Grupo de df_tasks que será a futura tabela fato
    df.drop(['StatusExterno', 'tmpStatusExterno'], axis=1, inplace=True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['StatusExterno'] == '', ['StatusExterno']] = '<sem status>'

    print('%d linhas OK.' %dfx.shape[0])
    
    return dfx

### DIM usuario

In [17]:
def Mt_Usuarios(df):

    print('dim_Usuario - iniciando montagem... ', end='')

    # trasnforma a coluna Usuario em uma coluna do tipo "category'
    df['tmpUsuario'] = df['Usuarios'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Usuario
    df['FK_dim_Usuarios'] = df['tmpUsuario'].cat.codes.astype('int64') + 1

    # tira a duplicidade
    dfx = df.loc[:, ['FK_dim_Usuarios', 'Usuarios']].drop_duplicates()
    col_ren = {'FK_dim_Usuarios': 'PK_dim_Usuarios', 'Usuarios': 'Usuario'}
    dfx.rename(columns=col_ren, inplace = True)

    # exclui a coluna Usuario de df_tasks que será a futura tabela fato
    df.drop(['Usuarios', 'tmpUsuario'], axis=1, inplace=True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['Usuario'] == '', ['Usuario']] = '<usuário indefinido>'

    print('%d linhas OK.' %dfx.shape[0])
    
    return dfx

### DIM situacao

In [18]:
def Mt_Situacao():

    print('dim_Situacao - iniciando montagem... ', end='')

    dfx = pd.DataFrame({'PK_dim_Situacao': [1, 2, 3],
                        'Situacao':        ['em Andamento', 'Encerrada', 'Cancelada']}
                        )
    print('%d linhas OK.' %dfx.shape[0])

    return dfx


### DIM SLA

In [19]:
def Mt_SLA():

    print('dim_SLA - iniciando montagem... ', end='')

    dfx = pd.DataFrame({'PK_dim_SLA': [1, 2, 3],
                        'DescSLA':    ['dentro do prazo', 'perto do prazo', 'fora do prazo'],
                        'Cor':        ['verde','amarelo' ,'vermelho']}
                    )
    print('%d linhas OK.' %dfx.shape[0])

    return dfx

### DIM endereco

In [20]:
def Mt_Endereco(dfe):

    print('dim_Endereco - iniciando montagem... ', end='')

    dc = ['state', 'city', 'neighborhood', 'zipcode', 'street']
    df = dfe.loc[dfe['atributo'].isin(dc), ['atributo','valor', 'protocolo']].copy()
# ===================================================    
    # faz pivot da coluna atributo
    df = df.pivot(columns='atributo', values='valor', index='protocolo').reset_index()
    df.columns.name = None

    # completa dataset com campos obrigatórios para composição de endereço 
    for i in dc:
        if i not in df.columns:
            df[i] = ''
    
    # renomeia colunas
    dic_renome = {'protocolo': 'Solicitacao', 
                  'zipcode': 'CEP', 
                  'street': 'Endereco', 
                  'neighborhood': 'Bairro', 
                  'city': 'Cidade', 
                  'state': 'UF'}
    df.rename(columns=dic_renome, inplace = True)
    
    # substitui a coluna Endereco por vazio quando '-' ou null
    df.loc[(df['Endereco'] == '-') | (df['Endereco'].isnull()), ['Endereco']] = ''
    df.loc[(df['Bairro'] == '-') | (df['Bairro'].isnull()), ['Bairro']] = ''

    # cria a coluna Endereco Completo
    df['EnderecoCompleto'] = df['Cidade'] + ', ' + df['UF'] + ', Brasil' 

# ===================================================    

    dict_uf = {'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas', 'BA': 'Bahia',
               'CE': 'Ceará', 'ES': 'Espírito Santo', 'GO': 'Goiás', 'MA': 'Maranhão', 'MT': 'Mato Grosso',
               'MS': 'Mato Grosso do Sul', 'MG': 'Minas Gerais', 'PA': 'Pará', 'PB': 'Paraíba',
               'PR': 'Paraná', 'PE': 'Pernambuco', 'PI': 'Piauí', 'RJ': 'Rio de Janeiro',
               'RN': 'Rio Grande do Norte', 'RS': 'Rio Grande do Sul', 'RO': 'Rondônia', 'RR': 'Roraima',
               'SC': 'Santa Catarina', 'SP': 'São Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins', 'DF': 'Distrito Federal',
               '<nd>': '<sem UF>'
    }

    # acrescenta o nome do estado ao dataframe
    dfx = df.copy()
    dfx['NomeUF'] = dfx['UF'].apply(lambda x: dict_uf.get(x, None))
    dfx.rename(columns={'Solicitacao': 'PK_dim_Endereco'}, inplace = True)

    # preenche colunas com linhas vazias
    dfx.loc[dfx['CEP'].isna(), ['CEP']] = ''

    print('%d linhas OK.' %dfx.shape[0])

    return dfx

### FAT tasks

In [21]:
# função retorna a situação da solicitação de acordo com as coilunas cancelado e encerrado
def RetSit(Enc, Canc):
    rt = 0
    if Canc == 1:
        rt = 3
    elif Enc == 1:
        rt = 2
    else:
        rt = 1
    return rt

In [22]:
def CalculaPrazoSolicitacao(df):
    # guarda a data atual
    AGORA = pd.Timestamp('today')

    # cria uma coluna temporária para calcular o prazo
    df['tmp_DataHora_Conclusao'] = df['DataHora_Conclusao']

    # na coluna temporária, se não houver data de conclusão colocar a datahora atual
    df.loc[df['tmp_DataHora_Conclusao'].isna(), 'tmp_DataHora_Conclusao'] = AGORA

    # calcula o prazo da tarefa em dias com decimal
    df['prazo_task'] = (df['tmp_DataHora_Conclusao'] - df['DataHora_Criacao']) / pd.to_timedelta(1, unit='D')

    # cria um dataframe temporário com a soma de prazos de uma solicitação
    dfx = df[['FK_dim_Solicitacoes', 'prazo_task']].groupby('FK_dim_Solicitacoes').sum().reset_index()

    # cria uma coluna de soma de prazo da respectiva solicitação
    df = df.merge(dfx, how='left', on='FK_dim_Solicitacoes')
    
    # cria um dataframe temporário com a data da solicitação (menor data entre as tasks)
    dfx = df[['FK_dim_Solicitacoes', 'DataHora_Criacao']].groupby('FK_dim_Solicitacoes').min().reset_index()
    dfx.columns = ['FK_dim_Solicitacoes', 'DataHora_Solicitacao']
    
    # cria uma coluna com a data da solicitação
    df = df.merge(dfx, how='left', on='FK_dim_Solicitacoes')
    
    df['DataSolicitacao'] = pd.to_datetime(df['DataHora_Solicitacao']).dt.date
    df['HoraSolicitacao'] = pd.to_datetime(df['DataHora_Solicitacao']).dt.time    

    # exclui a coluna GrupoResponsavel de df_tasks que será a futura tabela fato
    df.drop(['tmp_DataHora_Conclusao', 'DataHora_Solicitacao'], axis=1, inplace=True)
    df.rename(columns={'prazo_task_y': 'PrazoSolicitacao', 'prazo_task_x': 'PrazoTask'}, inplace = True)
    
    return df

In [23]:
def Mt_Tasks(df, dfr):

    print('fat_Tasks - iniciando montagem... ', end='')

    dfx = df.copy()
    
    # faz um merge com o dataset de ratings
    dfx = df.merge(dfr, left_on='Protocolo', right_on='Solicitacao', how='left')
    dfx.drop(['Solicitacao'], axis=1, inplace=True)

    # define o código de situação da solicitação
    df_aux = dfx[['Protocolo', 'ProcessoEncerrado', 'ProcessoCancelado']].drop_duplicates()

    # substui valores de colunas
    troca = {'false': 0, 'true': 1}
    df_aux['ProcessoEncerrado'] = df_aux['ProcessoEncerrado'].map(troca)
    df_aux['ProcessoCancelado'] = df_aux['ProcessoCancelado'].map(troca)

    # agrega as ações para um registro por protocolo
    df_aux = df_aux.groupby('Protocolo').agg({'ProcessoEncerrado': 'max', 'ProcessoCancelado': 'max'}).reset_index()

    # retorna a situção da solicitação: 1-em andamento, 2-encerrado, 3-cancelado
    df_aux['Situacao'] = df_aux.apply(lambda x: RetSit(x['ProcessoEncerrado'], x['ProcessoCancelado']), axis=1)
    df_aux.drop(['ProcessoEncerrado', 'ProcessoCancelado'], axis=1, inplace=True)

    # faz merge com o dataset de situação da solicitação
    dfx = dfx.merge(df_aux, left_on='Protocolo', right_on='Protocolo', how='left')
    dfx.drop(['ProcessoEncerrado', 'ProcessoCancelado'], axis=1, inplace=True)

    # coloca -1 para as solicitações que não possuem avaliação
    dfx.loc[dfx['NotaAvaliacao'].isna(), 'NotaAvaliacao'] = -1
    dfx['NotaAvaliacao'] = dfx['NotaAvaliacao'].astype('int32')

    # renomeia as colunas
    col_ren = {'Protocolo': 'FK_dim_Solicitacoes', 'Situacao': 'FK_dim_Situacao'}
    dfx.rename(columns=col_ren, inplace = True)

    # trata a coluna Prazo
    
    dfx.loc[dfx['Prazo'].isna(), 'Prazo'] = '0'
#     dfx.loc[(dfx['Prazo'] == '') | (dfx['Prazo'] == 'null'), 'Prazo'] = '0'
    dfx['Prazo'] = dfx['Prazo'].astype('int64')

    dfx['DataCriacao'] = pd.to_datetime(dfx['DataHora_Criacao']).dt.date
    dfx['HoraCriacao'] = pd.to_datetime(dfx['DataHora_Criacao']).dt.time

    dfx['DataConclusao'] = pd.to_datetime(dfx['DataHora_Conclusao']).dt.date
    dfx['HoraConclusao'] = pd.to_datetime(dfx['DataHora_Conclusao']).dt.time

    dfx = CalculaPrazoSolicitacao(dfx)

    cols_ordem = ['FK_dim_Solicitacoes', 'Prazo', 
                  'DataCriacao', 'HoraCriacao', 'DataHora_Criacao',
                  'DataConclusao', 'HoraConclusao', 'DataHora_Conclusao',
                  'NotaAvaliacao', 'MotivoAvaliacao', 
                  'DataAvaliacao', 'HoraAvaliacao', 
                  'DataSolicitacao', 'HoraSolicitacao',
                  'PrazoSolicitacao', 'PrazoTask',
                  'FK_dim_Entidades', 'FK_dim_Servicos', 'FK_dim_Usuarios', 'FK_dim_GruposUsuarios', 
                  'FK_dim_Acoes', 'FK_dim_StatusExt', 'FK_dim_CategoriasServicos', 'FK_dim_GrupoResponsavel', 
                  'FK_dim_MotivosCanc', 'FK_dim_Encaminhamento', 'FK_dim_Situacao']
    dfx = dfx[cols_ordem]

    print('%d linhas OK.' %dfx.shape[0])

    return dfx


### <font color='green'>META BI - Constantes</font>

In [24]:
# tabela de decisão de combinação Dimensões e Métricas
mtx =  [('STRING_DIM',  'INTEGER_MET', 'N', 'N', 'S', 'S'),
        ('STRING_DIM',  'DOUBLE_MET',  'N', 'N', 'S', 'S'),
        ('STRING_DIM',  'NULL_MET',    'S', 'S', 'N', 'N'),

        ('BOOLEAN_DIM', 'INTEGER_MET', 'S', 'N', 'N', 'N'),
        ('BOOLEAN_DIM', 'DOUBLE_MET',  'S', 'N', 'N', 'N'),
        ('BOOLEAN_DIM', 'NULL_MET',    'S', 'N', 'N', 'N'),

        ('INTEGER_DIM', 'INTEGER_MET', 'S', 'S', 'S', 'S'),
        ('INTEGER_DIM', 'DOUBLE_MET',  'N', 'N', 'N', 'N'),
        ('INTEGER_DIM', 'NULL_MET',    'S', 'S', 'N', 'N'),

        ('DOUBLE_MET',  'INTEGER_MET', 'N', 'N', 'N', 'N'),
        ('DOUBLE_MET',  'DOUBLE_MET',  'N', 'N', 'S', 'S'),
        ('DOUBLE_MET',  'NULL_MET',    'N', 'N', 'N', 'N'),

        ('ID_DIM',      'INTEGER_MET', 'N', 'N', 'S', 'S'),
        ('ID_DIM',      'DOUBLE_MET',  'N', 'N', 'S', 'S'),
        ('ID_DIM',      'NULL_MET',    'S', 'S', 'N', 'N')
        ]
# colunas da tabela de decisão de combinação Dimensões e Métricas
cols_mtx = ['t_dim', 't_met', 'Count', 'Distinct Count', 'Sum', 'Average']

### <font color='green'>META BI - Tabelas Auxiliares</font>

In [25]:
def GeraCodDimMet(d, m):
    d_dim = {'STRING_DIM': 10, 'BOOLEAN_DIM': 20, 'INTEGER_DIM' :30, 'DOUBLE_DIM': 40, 'ID_DIM': 50}
    d_met = {'INTEGER_MET': 1, 'DOUBLE_MET': 2, 'NULL_MET': 3}

    v_dim = d_dim[d] if d in d_dim else 0
    v_met = d_met[m] if m in d_met else 0
    
    return v_dim + v_met

In [26]:
def Enumerico(s, tipo):
    try:
        v = np.float64(s) if tipo == 'DOUBLE' else np.int64(s)
    except:
        return False
    
    return True

In [27]:
def Z_aux_entidade(df):
    # recebe df=tasks

    print('Z_aux_entidade - iniciando montagem... ', end='')

    df_ret = df[['Protocolo', 'Entidade']].drop_duplicates()
    df_ret.columns = ['protocolo', 'entidade']
    
    print('%d linhas OK.' %df_ret.shape[0])

    return df_ret

In [28]:
def Z_aux_BASE(df, df2):
    # recebe df=form, df2=df_aux_entidade

    print('Z_aux_BASE - iniciando montagem... ', end='')

    df_ret = df.copy()
    
    # transforma as strings atributo e campo relacionado em lower case
    df_ret[['atributo', 'campo_relacionado']] = df_ret[['atributo', 'campo_relacionado']].apply(lambda x: x.str.lower())

    # faz um merge da dimensão df_aux_BASE com a df_auxentidade para adicionar a coluna entidade
    # se não existir o protocolo na tasks não considera a linha -- isso é um erro de integridade da base de dados
    df_ret = df_ret.merge(df2, on='protocolo')    
    
    # cria 3 novas colunas vazias
    df_ret['MET_atributo'] = np.nan
    df_ret['MET_valor']    = np.nan
    df_ret['MET_tipo']     = np.nan

    # cria uma coluna como PK
    df_ret['PK'] = df_ret['entidade'] + '_' + df_ret['servico'] + '_' + \
                        df_ret['atributo'] + '_' + df_ret['protocolo']

    # cria uma coluna com o tipo de registro
    df_ret['TIPO_REG'] = 'BASE'
    
    print('%d linhas OK.' %df_ret.shape[0])
    
    return df_ret
    

In [29]:
# traramento do tipo INTEGER ou DOUBLE
def Z_aux_MET(df):
    # recebe df=df_aux_BASE

    print('Z_aux_MET e Z_aux_MET_Err - iniciando montagem... ', end='')

    # dataset de métricas - usa somente tipo "INTEGER" ou "DOUBLE"
    cols = ['atributo', 'nome', 'valor', 'protocolo', 'servico', 'tipo', 'campo_relacionado', 'entidade']
    df_ret = df.loc[(df['tipo'] == 'INTEGER') | (df['tipo'] == 'DOUBLE'), cols].copy()

    # os valores estão com formato: ponto para separador de milhar e vírgula para fração
    # tira os pontos separador de milhar e substitui vírgula por ponto
    df_ret['valor'] = df_ret['valor'].apply(lambda x: x.replace('.', '').replace(',', '.'))

    # alimenta uma coluna indicadora que mostra se o valor corresponde ou não ao tipo
    df_ret['numerico'] = df_ret.apply(lambda x: Enumerico(x['valor'], x['tipo']), axis=1)

    # cria um dataset de linhas cuja o valor não corresponde ao tipo
    df_ret_Err = df_ret[df_ret['numerico'] == False].drop('numerico', axis=1)

    # cria um dataset de linhas de métricas
    df_ret = df_ret[~df_ret['numerico'] == False].drop('numerico', axis=1)

    # identifica que esse é um dataset de métricas
    df_ret['TIPO_REG'] = 'MET'

    # exclui linhas que não seja uma métrica explícita
    df_ret = df_ret[~(df_ret['campo_relacionado'] == '')]

    # cria a coluna FK para relacionar com o dataset BASE
    df_ret['FK'] = df_ret['entidade'] + '_' + df_ret['servico'] + '_' + \
                   df_ret['campo_relacionado'] + '_' + df_ret['protocolo']

    # troca o nome de algumas colunas
    dc = {'atributo': 'MET_atributo', 'campo_relacionado': 'atributo', 'valor': 'MET_valor', 'tipo': 'MET_tipo'}
    df_ret.rename(columns=dc, inplace = True)

    # faz um merge com o dataset BASE 
    df_ret = df_ret.merge(df[['PK', 'valor', 'tipo']], left_on='FK', right_on='PK')
    df_ret.drop(['PK', 'FK'], axis=1, inplace=True)
    
    print('%d e %d linhas OK.' %(df_ret.shape[0], df_ret_Err.shape[0]))

    return df_ret, df_ret_Err
    

In [30]:
def Z_aux_DIM(df, df_m):
# recebe df=df_aux_BASE, df_m=df_aux_MET

    print('Z_aux_DIM - iniciando montagem... ', end='')

    # escolee as colunas que farão a cancatenação de dataframes
    cols = ['atributo', 'nome', 'valor', 'protocolo', 'servico', 'tipo', 
            'entidade', 'MET_atributo', 'MET_valor', 'MET_tipo', 'TIPO_REG']
    df_ret = df[cols].copy()
    df_ret = pd.concat([df_ret, df_m])
    
    print('%d linhas OK.' %df_ret.shape[0])

    return df_ret

In [31]:
def Z_aux_decisao():

    print('Z_aux_decisao - iniciando montagem... ', end='')

    df_ret = pd.DataFrame(mtx, columns=cols_mtx)
    df_ret['Cod_Decisao'] = df_ret.apply(lambda x: GeraCodDimMet(x['t_dim'], x['t_met']), axis=1)
    
    print('%d linhas OK.' %df_ret.shape[0])
    
    return df_ret

In [32]:
def Z_aux_DataSolicitacao(df):
    # recebe df=tasks

    print('Z_aux_DataSolicitacao - iniciando montagem... ', end='')

    df_tmp = df[['Protocolo', 'DataHora_Criacao']].copy()
    # cria uma coluna só de datas
    df_tmp['data_criacao'] = pd.to_datetime(df_tmp['DataHora_Criacao']).dt.date

    # pega a menor data de cada número de protocolo
    df_ret = df_tmp[['Protocolo', 'data_criacao']].groupby('Protocolo').agg('min').reset_index()
    df_ret.columns = ['protocolo', 'data_criacao']
    
    print('%d linhas OK.' %df_ret.shape[0])

    return df_ret


In [33]:
def Z_aux_pto(df):
    # recebe df=df_aux_dec

    print('Z_aux_pto - iniciando montagem... ', end='')

    # faz uma cópia da tabela de decisão com as colunas necessárias
    cols = ['Cod_Decisao'] + cols_mtx[2:]
    df_ret = df[cols].copy()

    # faz o unpivot
    df_ret = pd.melt(df_ret, id_vars=['Cod_Decisao'], 
                     value_vars=['Count', 'Distinct Count', 'Sum', 'Average'],
                     var_name='Atributo')
    df_ret = df_ret[df_ret['value'] == 'S'].drop('value', axis=1)

    # trasnforma a coluna Atributo em uma coluna do tipo "category'
    df_ret['Atributo'] = df_ret['Atributo'].astype('category')

    # cria uma nova coluna que será a coluna chave primária da dimensão Operação
    df_ret['FK_ZN_dim_OPERACAO'] = df_ret['Atributo'].cat.codes.astype('int64') + 1
    
    print('%d linhas OK.' %df_ret.shape[0])

    return df_ret

### <font color='green'>META BI - Tabelas Dim, Fat e Pto</font>

In [34]:
def Z_pto_Decisao_Operacao(df):
    # recebe df=df_Z_aux_pto

    print('Z_pto_decisao_operacao - iniciando montagem... ', end='')

    df_ret = df.copy()
    
    # cria a tabela df_pto_decisao_operacao necessária ao modelo dimensional
    df_ret = df_ret[['Cod_Decisao', 'FK_ZN_dim_OPERACAO']]
    df_ret.columns = ['FK_dim_DECISAO', 'FK_ZN_dim_OPERACAO']
    
    print('%d linhas OK.' %df_ret.shape[0])

    return df_ret


In [35]:
def Z_dim_DIM(df):
    # recebe df=df_aux_DIM

    print('Z_dim_DIM - iniciando montagem... ', end='')

    df_ret = df.copy()

    df_ret = df_ret[['atributo', 'nome']].drop_duplicates().reset_index(drop=True).reset_index()
    df_ret.columns = ['PK_Z_dim_DIM', 'dimensao', 'nome']
    df_ret['PK_Z_dim_DIM'] += 1
    
    print('%d linhas OK.' %df_ret.shape[0])

    return df_ret


In [36]:
def Z_dim_SERVICO(df):    
    # recebe df=df_aux_DIM

    print('Z_dim_SERVICO - iniciando montagem... ', end='')

    df_ret = df.copy()

    df_ret = df_ret['servico'].drop_duplicates().reset_index(drop=True).reset_index()
    df_ret.columns = ['PK_Z_dim_SERVICO', 'servico']
    df_ret['PK_Z_dim_SERVICO'] += 1
    
    print('%d linhas OK.' %df_ret.shape[0])

    return df_ret


In [37]:
def Z_dim_MET(df):    
    # recebe df=df_aux_DIM

    print('Z_dim_MET - iniciando montagem... ', end='')

    df_ret = df.copy()

    df_ret = df_ret.loc[~df_ret['MET_atributo'].isna(), 'MET_atributo'].drop_duplicates()
    df_ret = df_ret.reset_index(drop=True).reset_index()
    df_ret.columns = ['PK_Z_dim_MET', 'metrica']
    df_ret['PK_Z_dim_MET'] += 1

    print('%d linhas OK.' %df_ret.shape[0])

    return df_ret


In [38]:
def Z_dim_OPERACAO(df):    
    # recebe df=df_aux_pto

    print('Z_dim_OPERACAO - iniciando montagem... ', end='')

    df_ret = df.copy()

    df_ret = df_ret[['FK_ZN_dim_OPERACAO', 'Atributo']].drop_duplicates()
    df_ret.columns = ['PK_Z_dim_OPERACAO', 'operacao']

    print('%d linhas OK.' %df_ret.shape[0])

    return df_ret


In [39]:
def Z_dim_DECISAO(df):    
    # recebe df=df_aux_dec

    print('Z_dim_DECISAO - iniciando montagem... ', end='')

    df_ret = df.copy()

    df_ret = df_ret[['Cod_Decisao', 't_dim', 't_met']].copy().rename(columns={'Cod_Decisao': 'PK_Z_dim_DECISAO'})

    print('%d linhas OK.' %df_ret.shape[0])

    return df_ret


In [40]:
def Z_fat_META(df, df_dim, df_ser, df_met, df_dts):    
    # recebe df=df_Z_dim_DIM, df_ser=df_Z_dim_SERVICO, df_met= df_Z_dim_MET, df_dts=df_Z_aux_DataSolicitacao

    print('Z_fat_META - iniciando montagem... ', end='')

    # copia o df auxiliar
    df_ret = df.copy()

    # faz merge com dim_DIM
    df_ret = df_ret.merge(df_dim, 
                          how='left', 
                          left_on=['atributo', 'nome'], 
                          right_on=['dimensao', 'nome']).drop(['atributo', 'nome', 'dimensao'], axis=1)
    # faz merge com dim_SERVICO
    df_ret = df_ret.merge(df_ser, 
                          how='left', 
                          left_on='servico', 
                          right_on='servico').drop('servico', axis=1)
    # faz merge com dim_MET
    df_ret = df_ret.merge(df_met, 
                          how='left', 
                          left_on='MET_atributo', 
                          right_on='metrica').drop(['MET_atributo', 'metrica'], axis=1)
    # faz merge com Z_aux_DataSolicitacao
    df_ret = df_ret.merge(df_dts, 
                          how='left', 
                          on='protocolo') #.drop(['MET_atributo', 'metrica'], axis=1)

    # cria a coluna indicadora de NULL_MET
    df_ret['reg_nativo'] = df_ret['MET_tipo'].apply(lambda x: 1 if pd.isna(x) else 0)

    # passa as colunas de tipo para a string indicadora de NULL
    df_ret.loc[df_ret['MET_tipo'].isna(), 'MET_tipo'] = 'NULL_MET'
    df_ret.loc[df_ret['tipo'].isna(), 'tipo'] = 'NULL_DIM'


    # substitui valores da coluna tipo
    subs_dim = {'STRING':'STRING_DIM', 
                'INTEGER': 'INTEGER_DIM', 
                'BOOLEAN':'BOOLEAN_DIM', 
                'DOUBLE': 'DOUBLE_DIM',
                'ID':'ID_DIM'}
    subs_met = {'INTEGER': 'INTEGER_MET', 
                'DOUBLE': 'DOUBLE_MET'}

    df_ret['tipo'].replace(subs_dim, inplace=True)
    df_ret['MET_tipo'].replace(subs_met, inplace=True)

    # acha o código para a tabela de decisão da dupla de tipos como FK para a PK de "df_aux_dec"
    df_ret['FK_Z_dim_DECISAO'] = df_ret.apply(lambda x: GeraCodDimMet(x['tipo'], x['MET_tipo']), axis=1)

    # exclui colunas já usadas
    df_ret.drop(['tipo', 'MET_tipo'], axis=1, inplace=True)

    # acerta os nomes das colunas
    dc = {'PK_Z_dim_DIM': 'FK_Z_dim_DIM', 
          'PK_Z_dim_SERVICO': 'FK_Z_dim_SERVICO', 
          'PK_Z_dim_MET': 'FK_Z_dim_MET', 
          'data_criacao': 'FK_Z_Data_Criacao'}
    df_ret.rename(columns=dc, inplace = True)

    # acerta os tipos das colunas
    df_ret['FK_Z_dim_MET'] = df_ret['FK_Z_dim_MET'].astype('Int64')
    df_ret['MET_valor'] = df_ret['MET_valor'].astype('float64')
    df_ret['FK_Z_Data_Criacao'] = df_ret['FK_Z_Data_Criacao'].apply(pd.to_datetime)

    print('%d linhas OK.' %df_ret.shape[0])

    return df_ret


# <font color='red'>4.0 LOAD</font>

In [41]:
def Regras():
    tp_dfs = (
        (df_dim_Acao,               'dim_acoes',                 'replace'),               
        (df_dim_Categoria,          'dim_categorias_servicos',   'replace'), 
        (df_dim_Encaminhamento,     'dim_encaminhamento',        'replace'),      
        (df_dim_Entidade,           'dim_entidades',             'replace'),           
        (df_dim_GrupoResponsavel,   'dim_grupo_responsavel',     'replace'),   
        (df_dim_Grupo,              'dim_grupos_usuarios',       'replace'),     
        (df_dim_MotivoCanc,         'dim_motivos_canc',          'replace'),        
        (df_dim_Servicos,           'dim_servicos',              'replace'),            
        (df_dim_StatusExt,          'dim_status_ext',            'replace'),          
        (df_dim_Usuarios,           'dim_usuarios',              'replace'),            
        (df_dim_Situacao,           'dim_situacao',              'replace'),            
        (df_dim_SLA,                'dim_sla',                   'replace'),                 
        (df_dim_Endereco,           'dim_endereco',              'replace'),            
        (df_fat_tasks,              'fat_tasks',                 'replace'),               
        (df_Z_pto_decisao_operacao, 'z_pto_decisao_operacao',    'replace'),  
        (df_Z_dim_DIM,              'z_dim_dim',                 'replace'),  
        (df_Z_dim_SERVICO,          'z_dim_servico',             'replace'),  
        (df_Z_dim_MET,              'z_dim_met',                 'replace'),  
        (df_Z_dim_OPERACAO,         'z_dim_operacao',            'replace'),  
        (df_Z_dim_DECISAO,          'z_dim_decisao',             'replace'),  
        (df_Z_fat_META,             'z_fat_meta',                'replace'),
        (df_ts,                     'ctl_carga',                 'append')
    )
    return tp_dfs


In [42]:
def Load_BD(strAut, banco):

    # import the module
    # create sqlalchemy engine
    
    if banco == 'mysql':
        bd_load = "mysql+pymysql://{user}:{pw}@{host}/{db}"
    elif banco == 'postgresql':
        bd_load = "postgresql://{user}:{pw}@{host}/{db}"
    else:
        print('Nome de Banco de Dados {} Inválido'.format(banco));
        return 1

    print('LoadBD - carregando tabelas no BD {} ... '.format(banco))

    SQLengine = create_engine(bd_load
                           .format(user = strAut['My_user'],
                                   pw   = strAut['My_pw'],
                                   host = strAut['My_host'],
                                   db   = strAut['My_db']),
                                   pool_recycle=3600)
    dbConn = SQLengine.connect()

    try:
        start_time = time.time()
        for i in Regras():
            print('    carregando tabela {} ... '.format(i[1]), end='')
            i[0].to_sql(i[1], con=dbConn, if_exists=i[2], index=False, chunksize = 10000)
            print('OK {0: .2f}'.format((time.time() - start_time)))
        print('LoadBD - carga de tabelas OK')
        
    except ValueError as vx:
        print('ERROR -', vx)

    except Exception as ex: 
        print('EXCEPTION -', ex)

    else:
        print('Tabelas criadas com sucesso');  

    finally:
        dbConn.close()
        return 0

# <font color='red'>0.0 MAIN</font>

In [52]:
# monta as strings de conexão
def StrConn(sit):
    host = 'localhost'; user = 'gd'; pw = 'Cavaquinho@Dourado@6390'
    
    
    in_prd  = {'My_host': host, 'My_db': 'bd_fontes',        'My_user': user, 'My_pw': pw}
    out_prd = {'My_host': host, 'My_db': 'bd_dw',            'My_user': user, 'My_pw': pw}
    in_hom  = {'My_host': host, 'My_db': 'bd_hom_fontes',    'My_user': user, 'My_pw': pw}
    out_hom = {'My_host': host, 'My_db': 'bd_hom_dw',        'My_user': user, 'My_pw': pw}
    in_dev  = {'My_host': host, 'My_db': 'bd_dev_fontes',    'My_user': user, 'My_pw': pw}
    out_dev = {'My_host': host, 'My_db': 'bd_dev_dw',        'My_user': user, 'My_pw': pw}
    in_sim  = {'My_host': host, 'My_db': 'bd_sim_fontes',    'My_user': user, 'My_pw': pw}
    out_sim = {'My_host': host, 'My_db': 'bd_sim_dw',        'My_user': user, 'My_pw': pw}

    # retorna o ambiente configurado
    if sit == 0:
        rt_in  = in_prd
        rt_out = out_prd
    elif sit == 1:
        rt_in  = in_hom
        rt_out = out_hom
    elif sit == 2:
        rt_in  = in_dev
        rt_out = out_dev
    else:
        rt_in  = in_sim
        rt_out = out_sim

    return rt_in, rt_out

In [46]:
def gd_timestamp():
    lst = {'DataHora': [datetime.today()], 'Versao': [ETL_VERSAO]}
    df = pd.DataFrame(lst)
    
    return df

In [53]:
if __name__ == '__main__':
    
    # 0-produção, 1-homologação, 2-desenvolvimento, 3-simulação
    dic_tipo = {0: 'produção', 1: 'homologação', 2: 'desenvolvimento', 3: 'simulação'}
    etl_tipo = 3
    
    print(f'Iniciando ETL de {dic_tipo[etl_tipo]}')
#     print('Iniciando ETL {0}'.format(dic_tipo[etl_tipo]))
    
    # EXTRACT
    AUTENTIC_IN, AUTENTIC_OUT = StrConn(etl_tipo)
    df_tasks, df_sla, df_form, df_rating = LeFontes(AUTENTIC_IN)
    
    DF_TESTE = df_sla.copy()
    
    # TRANSFORM
    # trata os datasets
    df_sla    = trSLA(df_sla)
    df_form   = trForm(df_form)
    df_rating = trRating(df_rating)
    df_tasks  = trTasks(df_tasks)
    
    dft = df_tasks.copy()
    dff = df_form.copy()
    
    # monta as tabelas dimensão
    df_dim_Acao              = Mt_dim_Acoes(df_tasks)
    df_dim_Categoria         = Mt_dim_CategoriaServico(df_tasks)
    df_dim_Encaminhamento    = Mt_dim_Encaminhamento(df_tasks)
    df_dim_Entidade          = Mt_dim_Entidade(df_tasks)
    df_dim_GrupoResponsavel  = Mt_dim_GrupoResponsavel(df_tasks)
    df_dim_Grupo             = Mt_dim_GruposUsuarios(df_tasks)
    df_dim_MotivoCanc        = Mt_MotivosCanc(df_tasks)
    df_dim_Servicos          = Mt_Servico(df_tasks, df_sla)
    df_dim_StatusExt         = Mt_StatusExt(df_tasks)
    df_dim_Usuarios          = Mt_Usuarios(df_tasks)
    df_dim_Situacao          = Mt_Situacao()
    df_dim_SLA               = Mt_SLA()
    df_dim_Endereco          = Mt_Endereco(df_form)
    
    # monta a tabela fato
    df_fat_tasks             = Mt_Tasks(df_tasks, df_rating)
    
    # monta as tabelas auxiliares do META BI
    df_Z_aux_entidade              = Z_aux_entidade(dft)
    df_Z_aux_BASE                  = Z_aux_BASE(dff, df_Z_aux_entidade)
    df_Z_aux_MET, df_Z_aux_MET_Err = Z_aux_MET(df_Z_aux_BASE)
    df_Z_aux_DIM                   = Z_aux_DIM(df_Z_aux_BASE, df_Z_aux_MET)
    df_Z_aux_decisao               = Z_aux_decisao()
    df_Z_aux_DataSolicitacao       = Z_aux_DataSolicitacao(dft)
    df_Z_aux_pto                   = Z_aux_pto(df_Z_aux_decisao)
    
    # monta as tabelas dimensão do META BI
    df_Z_pto_decisao_operacao      = Z_pto_Decisao_Operacao(df_Z_aux_pto)
    df_Z_dim_DIM                   = Z_dim_DIM(df_Z_aux_DIM)
    df_Z_dim_SERVICO               = Z_dim_SERVICO(df_Z_aux_DIM)
    df_Z_dim_MET                   = Z_dim_MET(df_Z_aux_DIM)
    df_Z_dim_OPERACAO              = Z_dim_OPERACAO(df_Z_aux_pto)
    df_Z_dim_DECISAO               = Z_dim_DECISAO(df_Z_aux_decisao)

    # monta as tabelas dimensão do META BI
    df_Z_fat_META                  = Z_fat_META(df_Z_aux_DIM,
                                                df_Z_dim_DIM, 
                                                df_Z_dim_SERVICO, 
                                                df_Z_dim_MET, 
                                                df_Z_aux_DataSolicitacao)
    
    # monta data e hora e versão da atualização
    df_ts = gd_timestamp()

    # LOAD
    ret = Load_BD(AUTENTIC_OUT, 'mysql')
    ret = Load_BD(AUTENTIC_OUT, 'postgresql')
    

Iniciando ETL de simulação
Iniciando leitura de dados nativos...OK
tasks:3293075 registros lidos em 16 colunas
sla: 942 registros lidos em 5 colunas
form: 5665808 registros lidos em 7 colunas
rating: 211707 registros lidos em 4 colunas
MySQL connection is closed
df_sla - iniciando transformação... 159 linhas OK.
df_form - iniciando transformação... 5665808 linhas OK.
df_rating - iniciando transformação... 211707 linhas OK.
df_tasks - iniciando transformação... 3293075 linhas OK.
dim_Acoes - iniciando montagem... 101 linhas OK.
dim_Categoria - iniciando montagem... 71 linhas OK.
dim_Encaminhamento - iniciando montagem... 94 linhas OK.
dim_Entidade - iniciando montagem... 21 linhas OK.
dim_GrupoResponsavel - iniciando montagem... 55 linhas OK.
dim_GruposUsuarios - iniciando montagem... 49 linhas OK.
dim_MotivosCanc - iniciando montagem... 21 linhas OK.
dim_Servico - iniciando montagem... 159 linhas OK.
dim_StatusExt - iniciando montagem... 98 linhas OK.
dim_Usuario - iniciando montagem..

ModuleNotFoundError: No module named 'pymysql'

In [55]:
ret = Load_BD(AUTENTIC_OUT, 'mysql')

LoadBD - carregando tabelas no BD mysql ... 


OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'Dourado@6390@localhost' ([Errno 11003] getaddrinfo failed)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

# TESTE